# 第2回開発者コミュニティミートアップ 機械学習で手書き数字の識別に挑戦

このワークショップでは、MNISTが提供している手書き数字の画像データセットを使用し、手書きの数字を識別する分類器の作成を通して、機械学習の基本をハンズオン形式で学びます。このワークショップの内容は、以下の書籍を参考にしています。

> Hands-On Machine Learning with Scikit-Learn, Keras, and TensorFlow, Third Edition, O'Reilly Media Inc. 　
<br> (邦訳)scikit-learn、Keras、TensorFlowによる実践機械学習 第2版

## 必要なライブラリをインポート

In [11]:
import numpy as np
import pandas as pd
from sklearn.datasets import fetch_openml
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn.dummy import DummyClassifier
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier

import matplotlib.pyplot as plt

## MNISTのデータセットをダウンロード
ダウンロードされたデータを、変数mnistに格納しておきます。

In [28]:
print(mnist.DESCR)

**Author**: Yann LeCun, Corinna Cortes, Christopher J.C. Burges  
**Source**: [MNIST Website](http://yann.lecun.com/exdb/mnist/) - Date unknown  
**Please cite**:  

The MNIST database of handwritten digits with 784 features, raw data available at: http://yann.lecun.com/exdb/mnist/. It can be split in a training set of the first 60,000 examples, and a test set of 10,000 examples  

It is a subset of a larger set available from NIST. The digits have been size-normalized and centered in a fixed-size image. It is a good database for people who want to try learning techniques and pattern recognition methods on real-world data while spending minimal efforts on preprocessing and formatting. The original black and white (bilevel) images from NIST were size normalized to fit in a 20x20 pixel box while preserving their aspect ratio. The resulting images contain grey levels as a result of the anti-aliasing technique used by the normalization algorithm. the images were centered in a 28x28 image b

In [12]:
mnist = fetch_openml('mnist_784', as_frame=False)

## データの内容の確認
dataには、数字の画像データが入っています。データは、Numpyの2次元配列です。
mnist.data.shapeには、データの「形」が入っています。(70000, 784)は、700000文字分のデータが入っていて、各文字（数字）は784次元の配列で表されていることを確認します。

In [ ]:
mnist.data.shape

### 数字イメージのデータを表示
784要素は、28x28ピクセルのモノクロイメージを表しています。（白=0, 黒=255）
mnist.data[]の添え字を変えて、いろいろなデータの中身を表示してみてください。

In [ ]:
mnist.data[0]

### イメージを表示
次のコードの mnist.data[]] の添字に、0~69999 の数字を入れて、いろいろなイメージを確認してみてください。

In [ ]:
def plot_digit(image_data):
    image = image_data.reshape(28, 28)
    plt.imshow(image, cmap="binary")
    plt.axis("off")

some_digit = mnist.data[0]
plot_digit(some_digit)
plt.show()

### 正解データの確認
mnist.targetは70000要素のNumpy 1次元配列で、mnist.dataの画像イメージが実際どの数字であるか（正解データ、ラベル）が格納されています。mnist.data[0]の正解がmnist.target[0],...,mnist.data[69999]の正解がmnist.target[69999]というふうに対応づけられています。

例えば、mnist.data[55]のイメージの正解は、mnist.target[55]です（=8）。

In [ ]:
print(mnist.target.shape)
print(mnist.target)
print(mnist.target[55])

# 教師データの準備

機械学習では、説明変数（ここでは画像イメージ）をX、目的変数（正解データ）をyとすることが多いので、X, yにデータを代入しておきます。

この説明変数と目的変数のペアを教師データと呼びます。

In [17]:
X = mnist.data
y = mnist.target

## データの分割
データを、学習用とテスト用に分割します。ここでは、70000個のデータのうち、学習用に60000個、テスト用に10000個とします。

> *【Q】 データを学習用とテスト用に分割する理由はなんでしょう？*

In [18]:
X_train, X_test, y_train, y_test = X[:60000], X[60000:], y[:60000], y[60000:]

# 2値分類器の学習
0~9の数字のイメージを分類することは多値分類タスクです。数字が10種類あるので、10種類の中から一つの正解を選択するからです。

しかし、ここでは、学習の仕組みを理解するために、数字のイメージが5か5でないかを分類する2値分類タスクを題材とし、学習にまつわる事項を取り上げていきます。

まず、正解データを、5であればTrue、そうでなければFalseに変形し、y_train_5, y_test_5に格納します。

In [ ]:
y_train_5 = (y_train == '5')
y_test_5 = (y_test == '5')

print(y_train_5)

### SGD(確率的勾配降下法)を使って学習
ここでは、学習アルゴリズムとして、SGD確率勾配降下法を使って学習を行います。

SGD確率勾配降下法は、シンプルな線形分類器です。様々なパラメータが指定可能ですが、今回はデフォルト値を利用します。

Scikit learnライブラリのSGDClassifierクラスを使用すれば、学習の詳細を知らなくても容易に学習が可能です。fit()メソッドに学習用データと、それに対応する正解データを渡すだけで学習が実行できます。

In [ ]:
sgd_clf = SGDClassifier(random_state=42)
sgd_clf.fit(X_train, y_train_5)

### 予測と正解の比較

分類器が5だと予測したデータの番号と、正解データの中の5のデータの番号を比較してみましょう。画面の都合上、最初の100個のデータで比較します。

> *多くのデータで、予測と正解が一致しており、一部のデータでは一致していないことを確かめてみましょう。*

In [ ]:
print(np.where(sgd_clf.predict(X[0:1000])))
print(np.where(y[0:1000]=='5'))

### モデルの評価
モデルがどのくらい正確に予測を行っているかを定量的に把握することは重要です。機械学習の性能を測る指標には代表的なものがいくつかありますが、まずは、最もシンプルな指標である「精度(precision)」を算出します。精度は、予測が正解と一致したデータ数の全体に対する割合です。

cross_val_score()は、クロスバリデーション(交差検証)と呼ばれる手法で、複数回学習とテストを繰り返す関数です。cv=3とすることで、繰り返しを3回にしています。

くり返しの数だけ結果（精度）が返されます。

> *【Q】 返された精度は高いと言えるでしょうか？*

In [ ]:
cross_val_score(sgd_clf, X_train, y_train_5, cv=3, scoring="accuracy")

### ダミー分類器の性能
95%を超える精度は感覚的に高いと思うかもしれません。しかし、実際には精度だけで性能を評価することは良い方法ではありません。それを示すために、ダミー分類器を使います。ダミー分類器は、正解データのうち最も頻度の高い値（今回はFalse: 5ではない）を返します。

In [ ]:

dummy_clf = DummyClassifier()
dummy_clf.fit(X_train, y_train_5)
print(any(dummy_clf.predict(X_train)))  # prints False: no 5s detected

ダミー分類器でもクロスバリデーションで精度を測定してみます。

> 　*【Q】得られた精度について、なぜこのような（高い）値が出るのか考えてみましょう。*

In [ ]:
cross_val_score(dummy_clf, X_train, y_train_5, cv=3, scoring="accuracy")

## 混同行列を使った性能評価

予測が当たっているかどうかに着目する指標が精度です。しかし、ダミー分類器のように常にFalseと予測しても、正解がほとんどFalseであれば、高い精度が出てしまうのが問題です。

予測の当たり外れを詳細に考えてみると、予測が外れる時に、Trueが正解であるものにFalseと予測する場合と、Falseが正解である場合にTrueと予測する場合があることに気づきます。このような状況を整理するのに、混同行列が役に立ちます。

混同行列の説明は、[README.mdの説明](https://github.com/Intersystems-jp/meetup2024WorkShop/blob/main/3-c.ML101/README.md#混同行列)を見てください。

In [66]:

y_train_pred = cross_val_predict(sgd_clf, X_train, y_train_5, cv=3)

In [ ]:
cm = confusion_matrix(y_train_5, y_train_pred)
cm

In [ ]:
pd.DataFrame(classification_report(y_train_5, y_train_pred, output_dict=True)).transpose()

In [ ]:
y_train_pred_dummy = cross_val_predict(dummy_clf, X_train, y_train_5, cv=3)
cm = confusion_matrix(y_train_5, y_train_pred_dummy)
cm

In [ ]:
pd.DataFrame(classification_report(y_train_5, y_train_pred_dummy, output_dict=True)).transpose()

In [ ]:
sgd_clf.decision_function(X_train[0:100])

In [52]:
y_scores = cross_val_predict(sgd_clf, X_train, y_train_5, cv=3,
                             method="decision_function")

In [53]:
from sklearn.metrics import precision_recall_curve

precisions, recalls, thresholds = precision_recall_curve(y_train_5, y_scores)

In [ ]:
threshold = 0
plt.plot(thresholds, precisions[:-1], "b--", label="Precision", linewidth=2)
plt.plot(thresholds, recalls[:-1], "g-", label="Recall", linewidth=2)
plt.vlines(threshold, 0, 1.0, "k", "dotted", label="threshold")
idx = (thresholds >= threshold).argmax()  # first index ≥ threshold
plt.plot(thresholds[idx], precisions[idx], "bo")
plt.plot(thresholds[idx], recalls[idx], "go")
plt.axis([-50000, 50000, 0, 1])
plt.grid()
plt.xlabel("Threshold")
plt.legend(loc="center right")
plt.show()

In [ ]:
y_test_pred = sgd_clf.predict(X_test)
pd.DataFrame(classification_report(y_test_5, y_test_pred, output_dict=True)).transpose()

In [ ]:

svm_clf = SVC(random_state=42)
svm_clf.fit(X_train, y_train) 

In [ ]:
print(svm_clf.predict(X[0:50]))
print(y[0:50])

In [ ]:
forest_clf = RandomForestClassifier(random_state=42)
y_train_pred_forest = cross_val_predict(forest_clf, X_train, y_train_5, cv=3)
cm = confusion_matrix(y_train_5, y_train_pred_forest)
cm

In [ ]:
pd.DataFrame(classification_report(y_train_5, y_train_pred_forest, output_dict=True)).transpose()

In [ ]:
forest_clf.fit(X_train, y_train_5)
y_test_pred_forest = forest_clf.predict(X_test)
pd.DataFrame(classification_report(y_test_5, y_test_pred_forest, output_dict=True)).transpose()